In [ ]:
!rm -rf sample_data
!git clone https://www.github.com/harvitronix/five-video-classification-methods.git
!mv five-video-classification-methods/* .
!rm -rf five-video-classification-methods

Cloning into 'five-video-classification-methods'...
remote: Enumerating objects: 495, done.
remote: Total 495 (delta 0), reused 0 (delta 0), pack-reused 495
Receiving objects: 100% (495/495), 322.37 KiB | 3.88 MiB/s, done.
Resolving deltas: 100% (289/289), done.


In [ ]:
!rsync --progress drive/MyDrive/data_tar/test.tar .
!tar xf test.tar --checkpoint=.5000
!rm test.tar
!mv test data
!echo

!rsync --progress drive/MyDrive/data_tar/train.tar .
!tar xf train.tar --checkpoint=.10000
!rm train.tar
!mv train data

test.tar
  5,118,146,560 100%   21.20MB/s    0:03:50 (xfr#1, to-chk=0/1)
...................................................................................................
train.tar
 13,107,425,280 100%   18.82MB/s    0:11:04 (xfr#1, to-chk=0/1)
................................................................................................................................

In [ ]:
!mkdir data/checkpoints
!mkdir data/logs

In [ ]:
# replace the csv file with the one from drive
!mv data/data_file.csv data/data_file_original.csv
!rsync -progress drive/MyDrive/data/data_file.csv data/data_file.csv

In [ ]:
# TRAIN CNN

from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, CSVLogger
from data import DataSet
import os.path
import time

data = DataSet()

# Helper: Save the model.
checkpointer = ModelCheckpoint(
    filepath=os.path.join('data', 'checkpoints', 'inception.{epoch:03d}-{val_loss:.2f}.hdf5'),
    verbose=1,
    save_best_only=True)

# Helper: Stop when we stop learning.
early_stopper = EarlyStopping(patience=10)

# Helper: TensorBoard
tensorboard = TensorBoard(log_dir=os.path.join('data', 'logs'))

# Helper: CSV Logger
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('data', 'logs', 'cnn-training-' + \
        str(timestamp) + '.log'))

def get_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        horizontal_flip=True,
        rotation_range=10.,
        width_shift_range=0.2,
        height_shift_range=0.2)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        os.path.join('data', 'train'),
        target_size=(299, 299),
        batch_size=32,
        classes=data.classes,
        class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
        os.path.join('data', 'test'),
        target_size=(299, 299),
        batch_size=32,
        classes=data.classes,
        class_mode='categorical')

    return train_generator, validation_generator

def get_model(weights='imagenet'):
    # create the base pre-trained model
    base_model = InceptionV3(weights=weights, include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer
    predictions = Dense(len(data.classes), activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

def freeze_all_but_top(model):
    """Used to train just the top layers of the model."""
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in model.layers[:-2]:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def freeze_all_but_mid_and_top(model):
    """After we fine-tune the dense layers, train deeper."""
    # we chose to train the top 2 inception blocks, i.e. we will freeze
    # the first 172 layers and unfreeze the rest:
    for layer in model.layers[:172]:
        layer.trainable = False
    for layer in model.layers[172:]:
        layer.trainable = True

    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    model.compile(
        optimizer=SGD(lr=0.0001, momentum=0.9),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_k_categorical_accuracy'])

    return model

def train_model(model, nb_epoch, generators, callbacks=[]):
    train_generator, validation_generator = generators
    model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        validation_data=validation_generator,
        validation_steps=10,
        epochs=nb_epoch,
        callbacks=callbacks)
    return model

def main(weights_file):
    model = get_model()
    generators = get_generators()

    if weights_file is None:
        print("Loading network from ImageNet weights.")
        # Get and train the top layers.
        model = freeze_all_but_top(model)
        model = train_model(model, 10, generators)
    else:
        print("Loading saved model: %s." % weights_file)
        model.load_weights(weights_file)

    # Get and train the mid layers.
    model = freeze_all_but_mid_and_top(model)
    model = train_model(model, 1000, generators,
                        [checkpointer, early_stopper, tensorboard, csv_logger])

main(weights_file=None)

87916544/87910968 [==============================] - 1s 0us/step
Found 1788425 images belonging to 101 classes.
Found 697865 images belonging to 101 classes.
Loading network from ImageNet weights.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
100/100 [==============================] - 89s 742ms/step - loss: 5.3521 - accuracy: 0.1094 - val_loss: 2.6312 - val_accuracy: 0.4062
Epoch 2/10
100/100 [==============================] - 77s 741ms/step - loss: 2.4917 - accuracy: 0.4036 - val_loss: 2.1678 - val_accuracy: 0.4563
Epoch 3/10
100/100 [==============================] - 82s 751ms/step - loss: 1.9053 - accuracy: 0.5268 - val_loss: 1.6559 - val_accuracy: 0.5312
Epoch 4/10
100/100 [==============================] - 80s 735ms/step - loss: 1.5193 - accuracy: 0.5943 - val_loss: 1.7825 - val_accuracy: 0.5156
Epoch 5/10
100/100 [==============================] - 80s 735ms/step - loss: 1.3753 - accuracy: 0.6273 - val_loss: 1.6801 - val_accuracy: 0.5625
Epoch 6/10
100/100 [==============================] - 80s 743ms/step - loss: 1.2486 - accuracy: 0.6576 - val_loss: 1.7615 - val_accuracy: 0.5469
Epoch 7/10
100/100 [==============================] - 80s 736ms/step - loss: 1.2160 - accuracy: 0.6676 - val_loss: 1.7308 - val_ac

In [ ]:
!ls data/logs

cnn-training-1612368312.2867029.log  train  validation


In [ ]:
!cd data && zip logger6-cnn.zip checkpoints/inception.013-1.11.hdf5
!cd data && zip -r logger6-cnn.zip logs
!rsync data/logger6-cnn.zip drive/MyDrive/write_data/

  adding: checkpoints/inception.013-1.11.hdf5 (deflated 10%)
  adding: logs/ (stored 0%)
  adding: logs/cnn-training-1612368312.2867029.log (deflated 61%)
  adding: logs/validation/ (stored 0%)
  adding: logs/validation/events.out.tfevents.1612369317.04193ea7c4ae.62.25098.v2 (deflated 68%)
  adding: logs/train/ (stored 0%)
  adding: logs/train/plugins/ (stored 0%)
  adding: logs/train/plugins/profile/ (stored 0%)
  adding: logs/train/plugins/profile/2021_02_03_16_20_48/ (stored 0%)
  adding: logs/train/plugins/profile/2021_02_03_16_20_48/04193ea7c4ae.trace.json.gz (deflated 3%)
  adding: logs/train/plugins/profile/2021_02_03_16_20_48/04193ea7c4ae.input_pipeline.pb (deflated 56%)
  adding: logs/train/plugins/profile/2021_02_03_16_20_48/04193ea7c4ae.xplane.pb (deflated 80%)
  adding: logs/train/plugins/profile/2021_02_03_16_20_48/04193ea7c4ae.memory_profile.json.gz (deflated 0%)
  adding: logs/train/plugins/profile/2021_02_03_16_20_48/04193ea7c4ae.tensorflow_stats.pb (deflated 71%)
  add

In [ ]:
import sys
import csv
import matplotlib.pyplot as plt

training_log = '/content/data/logs/cnn-training-1609571960.010799.log'

validation_accuracies = []
validation_loss = []
training_accuracies = []
training_loss = []
validation_top_5_acc = []
training_top_5_acc = []

with open(training_log) as fin:
    reader = csv.reader(fin)
    next(reader, None)  # skip the header
    for epoch,acc,loss,top_k_categorical_accuracy,val_acc,val_loss,val_top_k_categorical_accuracy in reader:
        validation_accuracies.append(float(val_acc))
        validation_loss.append(float(val_loss))
        validation_top_5_acc.append(float(val_top_k_categorical_accuracy))
        training_loss.append(float(loss))
        training_accuracies.append(float(acc))
        training_top_5_acc.append(float(top_k_categorical_accuracy))

In [ ]:
plt.subplot(3, 1, 1)   
plt.plot(validation_loss, label='validation loss')
plt.plot(training_loss, label='training loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.xlim((0, len(training_loss)))
plt.legend()
plt.grid()
plt.savefig('cnn-metrics-plot-loss.png')
plt.title('Loss')

plt.subplot(3, 1, 2)
plt.plot(validation_accuracies, label='validation acc')
plt.plot(training_accuracies, label='training acc')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.xlim((0, len(training_accuracies)))
plt.legend()
plt.grid()
plt.savefig('cnn-metrics-plot-acc.png')
plt.title('Accuracy')

plt.subplot(3, 1, 3)
plt.plot(validation_top_5_acc, label='validation top-5 acc')
plt.plot(training_top_5_acc, label='training top-5 acc')
plt.xlabel('epochs')
plt.ylabel('top 5 categorical accuracy')
plt.xlim((0, len(training_accuracies)))
plt.legend()
plt.grid()
plt.savefig('cnn-metrics-plot-top5.png')
plt.title('Top 5 Categorical Accuracy')

plt.show()

In [ ]:
# function ConnectButton(){
#     console.log("Connect pushed"); 
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
# }
# interval = setInterval(ConnectButton,60000);